In [12]:
from numpy import array
from math import sqrt
import numpy as np
"""importing the necessary clustering libraries"""
from pyspark.mllib.clustering import KMeans, KMeansModel

# Load and parse the data
#data = sc.textFile("data/mllib/kmeans_data.txt")
"""Retrieving the Datasets from HDFS and filtering the columns that contain statistics of the batsman"""
parsedData = sc.textFile("hdfs://localhost:54310/project/batsmen_stat.csv") \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1 and line[1]!="Name") \
    .map(lambda line: array([float(line[2]),float(line[3]),float(line[4]),float(line[5]),\
                             float(line[6]),float(line[7]),float(line[8]),float(line[9]),float(line[10])]))

#l = parsedData1.map(lambda x:array([x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12]])).collect()
#parsedData = sc.parallelize(l)

#and line[3]!='Inns'
'''
parsedData = sc.textFile("batsman.csv") \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1) \
    .map(lambda line: array([line[1],line[2],line[3],line[4],line[5]]))
'''

#l = [x for x in parsedData.toLocalIterator()]
#l = [x.astype(np.float) for x in l[1:]]
#parsedData = sc.parallelize(l)


#parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 10, maxIterations=100000, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "hdfs://localhost:54310/project/bat_result")
sameModel = KMeansModel.load(sc, "hdfs://localhost:54310/project/bat_result")

print("Raina class :")
print(sameModel.predict(array([16,399,321,26.6,124.2990654206,15,1,39,10])))

print("Bumrah class :")
print(sameModel.predict(array([3,8,11,8,72.7272727273,1,2,1,0])))




Within Set Sum of Squared Error = 4476.613557267382
Raina class :
4
Bumrah class :
1


In [13]:
print("Virat Kohli :")
print(sameModel.predict(array([16,973,655,81.0833333333,148.5496183206,12,4,84,38])))

print("MS Dhoni:")
print(sameModel.predict(array([14,284,220,35.5,129.0909090909,8,6,18,14])))


names = sc.textFile("hdfs://localhost:54310/project/batsmen_stat.csv") \
    .map(lambda line: line.split(",")).filter(lambda line: len(line)>1 and line[1]!="Name").map(lambda line: array([line[1]]))
names = names.map(lambda x:x[0])
#print(names.collect())


player_class = parsedData.map(lambda x: (x,sameModel.predict(x)))
#print(player_class.collect())

name_class = names.zip(player_class)
#print(name_class.collect())


def players_of_class(k):
    print("Players of class ",k)
    k_class_players = name_class.map(lambda x:(x[0],x[1][1])).filter(lambda x:x[1]==k)
    a = k_class_players.collect()
    l = [i[0] for i in a]
    print("Number of players under class "+str(k)+" = ",len(a))
    print("\n\n")
    print(l)
    print("\n\n")
    #for i in a:
    #    print(i)

    
for i in range(10):
    players_of_class(i)





Virat Kohli :
5
MS Dhoni:
3
Players of class  0
Number of players under class 0 =  6



['AM Rahane', 'G Gambhir', 'M Vijay', 'Q de Kock', 'RG Sharma', 'S Dhawan']



Players of class  1
Number of players under class 1 =  35



['A Nehra', 'AD Nath', 'Ankit Sharma', 'BB Sran', 'C Munro', 'CA Lynn', 'CJ Jordan', 'D Wiese', 'F Behardien', 'HH Pandya', 'IK Pathan', 'Imran Tahir', 'Ishan Kishan', 'JJ Bumrah', 'JO Holder', 'KC Cariappa', 'KV Sharma', 'KW Richardson', 'LMP Simmons', 'M Morkel', 'MA Agarwal', 'MJ Guptill', 'MM Sharma', 'MR Marsh', 'NM Coulter-Nile', 'NS Naik', 'P Kumar', 'P Negi', 'PP Chawla', 'Parvez Rasool', 'R Ashwin', 'R Sathish', 'R Vinay Kumar', 'SS Iyer', 'Sandeep Sharma']



Players of class  2
Number of players under class 2 =  18



['AD Russell', 'CH Morris', 'DA Miller', 'DJ Hooda', 'GJ Maxwell', 'HM Amla', 'JP Duminy', 'M Vohra', 'MC Henriques', 'MP Stoinis', 'NLTC Perera', 'NV Ojha', 'PA Patel', 'RR Pant', 'SA Yadav', 'SE Marsh', 'SR Watson', 'SS Tiwary']



Pla